run with `popstrucure` env

In [1]:
import os
import vcf
from pathlib import Path
from matplotlib import pyplot as plt
from matplotlib import cm
import pandas as pd
from matplotlib.pyplot import figure
import glob
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import statistics
import allel
from collections import defaultdict
from itertools import combinations
from haversine import haversine
from skbio.stats.distance import mantel
from scipy.stats import ttest_ind
import cartopy.crs as ccrs
import cartopy.feature as cf
from pykrige.ok import OrdinaryKriging

Unable to determine R home: [Errno 2] No such file or directory: 'R'


In [2]:
proj_dir="/master/nplatt/sch_hae_scan"
results_dir="{}/results".format(proj_dir)

os.chdir(proj_dir)

In [3]:
info_df=pd.read_csv("{}/data/seq_and_sample_docs/all_sh_sb_sample_data.csv".format(proj_dir), sep=",") 

# Genome Coverage

In [4]:
Path("{}/genome_coverage".format(results_dir)).mkdir(parents=True, exist_ok=True)
os.chdir("{}/genome_coverage".format(results_dir))

In [5]:
pca_df=pd.read_csv("../initial_pca/pca_df.csv")

In [6]:
pca_df

,sample_name,1,2,3,4,5,6,7,8,9,...,locale,host,phenotype,project,data_type,lat,lon,kmeans_group,pca_species,kmeans_label
0,Sb_NG_au_1.2,-0.174693,0.025622,0.001506,0.004149,0.035574,0.055396,0.032718,-0.019520,0.007569,...,na,cow,bovis,nigeria,genome,7.067448,6.271139,2,s_bovis,sb
1,Sb_NG_au_2.10,-0.173848,0.025251,0.001886,0.005161,0.036499,0.060013,0.032135,-0.019021,0.003526,...,na,cow,bovis,nigeria,genome,7.067448,6.271139,2,s_bovis,sb
2,Sb_NG_au_2.13,-0.174493,0.025294,0.001699,0.001562,0.037332,0.060250,0.030712,-0.022199,0.003399,...,na,cow,bovis,nigeria,genome,7.067448,6.271139,2,s_bovis,sb
3,Sb_NG_au_2.5,-0.173862,0.025171,0.003061,-0.001638,0.037943,0.059797,0.031391,-0.015780,0.000577,...,na,cow,bovis,nigeria,genome,7.067448,6.271139,2,s_bovis,sb
4,Sb_NG_au_2.6,-0.173011,0.025618,0.001527,0.005455,0.038614,0.061741,0.032516,-0.022343,0.009809,...,na,cow,bovis,nigeria,genome,7.067448,6.271139,2,s_bovis,sb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,ssp_niger_libore_167,0.023482,-0.078226,-0.011342,-0.059942,0.017520,-0.043266,0.041449,-0.016997,0.021684,...,na,snail,unk,scan_africa,genome,13.390080,2.197110,0,s_haematobium,sh_nw
167,ssp_niger_libore_168,0.022237,-0.077112,-0.019412,-0.059978,0.006374,-0.004766,0.030555,-0.041763,0.080463,...,na,snail,unk,scan_africa,genome,13.390080,2.197110,0,s_haematobium,sh_nw
168,ssp_niger_libore_169,0.023857,-0.073894,-0.020205,-0.050080,0.015119,-0.015182,0.038133,-0.019263,0.036911,...,na,snail,unk,scan_africa,genome,13.404170,2.188020,0,s_haematobium,sh_nw
169,ssp_niger_libore_173,0.026630,-0.083945,-0.003698,-0.069531,-0.009745,-0.085123,0.095988,-0.108937,0.082528,...,na,snail,unk,scan_africa,genome,13.404170,2.188020,0,s_haematobium,sh_nw


In [11]:
samples = pca_df[~pca_df['sample_name'].isin(public_samples)]["sample_name"]

In [12]:
samples.to_csv("samples.list", header=None, index=False)

In [67]:
%%bash

QSUB="qsub -V -cwd -S /bin/bash -q all.q -j y -p -1023 -pe smp 24" 
REF="/master/nplatt/sch_hae_scan/data/GCF_000699445.3_UoM_Shae.V3_genomic.fna"

for SAMPLE in $(cat samples.list); do
    
    CRAM="/master/nplatt/sch_hae_scan/results/mapped_reads/${SAMPLE}_processed.cram"
    CMD="conda run -n popstructure --cwd . mosdepth --use-median --threads 4 --fasta ${REF} ${SAMPLE} ${CRAM}"
    
    echo $CMD | $QSUB -N ${SAMPLE} -o ${SAMPLE}.log
    
done

Your job 813290 ("Sb_NG_au_1.2") has been submitted
Your job 813291 ("Sb_NG_au_2.10") has been submitted
Your job 813292 ("Sb_NG_au_2.13") has been submitted
Your job 813293 ("Sb_NG_au_2.5") has been submitted
Your job 813294 ("Sb_NG_au_2.6") has been submitted
Your job 813295 ("Sb_NG_be_1.10") has been submitted
Your job 813296 ("Sb_NG_be_1.3") has been submitted
Your job 813297 ("Sb_NG_be_1.5") has been submitted
Your job 813298 ("Sb_NG_en_1.1") has been submitted
Your job 813299 ("Sh_NG_eb_6_2") has been submitted
Your job 813300 ("Sh_NG_ed_1_3") has been submitted
Your job 813301 ("Sh_NG_ed_3_2") has been submitted
Your job 813302 ("Sh_NG_kb_2_1") has been submitted
Your job 813303 ("Sh_NG_kw_1_10") has been submitted
Your job 813304 ("Sh_NG_os_1_4") has been submitted
Your job 813305 ("sbo_cdivoire_foro_106") has been submitted
Your job 813306 ("sbo_cdivoire_foro_116") has been submitted
Your job 813307 ("sbo_cdivoire_noumousso_107") has been submitted
Your job 813308 ("sbo_cdivoi

In [13]:
mean_covs=[]

for sample in samples:
    with open(f"{sample}.mosdepth.summary.txt", 'r') as f:
        for line in f:
            pass
        last_line = line
    
        mean_covs.append(float(last_line.split("\t")[3]))      

In [14]:
df=pd.DataFrame( {
    "sample": samples,
    "cov": mean_covs
})

df["cov"].median()

29.7

In [16]:
df.to_csv("genome_cov.csv", header=True, index=False, sep=",")
df

,sample,cov
0,Sb_NG_au_1.2,20.53
1,Sb_NG_au_2.10,23.15
2,Sb_NG_au_2.13,23.30
3,Sb_NG_au_2.5,24.73
4,Sb_NG_au_2.6,27.05
...,...,...
166,ssp_niger_libore_167,31.23
167,ssp_niger_libore_168,39.19
168,ssp_niger_libore_169,50.42
169,ssp_niger_libore_173,41.31
